In [1]:
# packages
using LinearAlgebra
using Random
Random.seed!();
using BenchmarkTools
using Optim
using Zygote

In [ ]:
# Tool fuctions
function symmetrize(A::AbstractArray)
    return (A + A')/2
end

In [2]:
struct cMPS{T<:AbstractArray}
    Q::T
    R::T
end

struct cMPO{T<:AbstractArray}
    Q::T  # onsite
    R::T  # interaction
    L::T  # interaction
    P::T  # long-range
end

In [3]:
# Products
function myprod(O::cMPO, S::cMPS)
    Oi = Matrix(1.0I,size(O.Q))
    Si = Matrix(1.0I,size(S.Q))
    Q = kron(Oi , S.Q) + kron(O.Q , Si) + kron(O.L , S.R)
    R = kron(O.R , Si) + kron(O.P , S.R)
    return cMPS(Q, R)
end

function myinnerprod(sl::cMPS, sr::cMPS, β::Real)
    li = Matrix(1.0I,size(sl.Q))    
    ri = Matrix(1.0I,size(sr.Q))    
    K = kron(li , sr.Q) + kron(sl.Q , ri) + kron(sl.R, sr.R)
    vals = eigvals(K)
    res = 0.
    for i=1:length(vals)
        res += exp(-β*vals[i])
    end
    return res
end

# Physical
function F(ψ::cMPS, W::cMPO, β::Real)
    Hψ = myprod(W,ψ)
    res = log(myinnerprod(ψ, Hψ ,β))- log(myinnerprod(ψ,ψ,β))
    return -1/β * res
end 

function difference(ψ1::cMPS, ψ2::cMPS; β=1)
    res = myinnerprod(ψ1,ψ1,β) + myinnerprod(ψ2,ψ2,β)
    res -= myinnerprod(ψ2,ψ1,β) + myinnerprod(ψ1,ψ2,β)
    return res
end

difference (generic function with 1 method)

\section{Transverse field Ising model(TFIM)}

Model Hamiltonian:
\begin{equation}
    H = \sum_{<ij>} -J Z_i Z_j -\sum_i \Gamma X_i
\end{equation}
where in $X$ and $Z$ are Pauli matirces.


In [4]:
J = 1.0; Γ = 1.0
X = [0 1; 1 0]
Z = [1 0; -1 0]
β = 1

W = cMPO(Γ*X, √J*Z, √J*Z, zeros(2,2))
Q = rand(2,2)
R = rand(2,2);
ψ = cMPS(Q,R)

cMPS{Array{Float64,2}}([0.8831879433987382 0.40860262057944774; 0.4092469733853561 0.4163402363076174], [0.24349282281502127 0.34449729704270426; 0.9005631021181324 0.8263147558380028])

In [ ]:
function OptimF(x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    return F(ψ,W,β)
end

function OptimF!(gx::Array{Float64,3}, x::Array{Float64,3})
    ψ = cMPS(x[:,:,1], x[:,:,2])
    grad = gradient(ψ -> F(ψ, W, β), ψ)[1]
    grad = cMPS(grad.Q, grad.R)
    gx = toarray(grad)
    return gx
end

function toarray(ψ::cMPS)
    # size(Q) == size(R)
    (r,c) = size(ψ.Q)
    x = zeros(r,c,2)
    x[:,:,1] = ψ.Q
    x[:,:,2] = ψ.R
    return x
end

In [ ]:
x = toarray(ψ) 
optimize(OptimF, OptimF!, x, LBFGS())

In [5]:
F(ψ,W,1)

-0.8173664918047276